In [18]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
import pandas as pd
from torchvision import transforms, models
from torch.utils.data import DataLoader
from PIL import Image
from sklearn.model_selection import train_test_split

In [13]:
import os
import csv

# Set the directory where your images are stored
root_dir = 'Dataset\Dataset\Training_data'  # Replace with the path to your images directory
output_csv = 'animal_images.csv'  # Name of the output CSV file

# List to hold all (image path, label) tuples
image_paths = []

# Walk through the directory and gather data
for root, dirs, files in os.walk(root_dir):
    for file in files:
        if file.lower().endswith(('.png', '.jpg', '.jpeg')):  # Check for image files
            print(root)
            label = os.path.basename(root)  # Folder name as label
            path = os.path.join(root, file)  # Full path to the image
            image_paths.append((path, label))

# Write data to CSV
with open(output_csv, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['image_path', 'label'])  # Header row
    for path, label in image_paths:
        writer.writerow([path, label])




Dataset\Dataset\Training_data\badger
Dataset\Dataset\Training_data\badger
Dataset\Dataset\Training_data\badger
Dataset\Dataset\Training_data\badger
Dataset\Dataset\Training_data\badger
Dataset\Dataset\Training_data\badger
Dataset\Dataset\Training_data\badger
Dataset\Dataset\Training_data\badger
Dataset\Dataset\Training_data\badger
Dataset\Dataset\Training_data\badger
Dataset\Dataset\Training_data\badger
Dataset\Dataset\Training_data\badger
Dataset\Dataset\Training_data\badger
Dataset\Dataset\Training_data\badger
Dataset\Dataset\Training_data\badger
Dataset\Dataset\Training_data\badger
Dataset\Dataset\Training_data\badger
Dataset\Dataset\Training_data\badger
Dataset\Dataset\Training_data\badger
Dataset\Dataset\Training_data\badger
Dataset\Dataset\Training_data\badger
Dataset\Dataset\Training_data\badger
Dataset\Dataset\Training_data\badger
Dataset\Dataset\Training_data\badger
Dataset\Dataset\Training_data\badger
Dataset\Dataset\Training_data\badger
Dataset\Dataset\Training_data\badger
D

In [25]:
class AnimalDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_name = self.dataframe.iloc[idx, 0]
        image = Image.open(img_name)
        label = self.dataframe.iloc[idx, 1]

        if self.transform:
            image = self.transform(image)

        # Convert label to torch.long
        label = torch.tensor(label, dtype=torch.long)

        return image, label


In [26]:
# Parameters
num_classes = 6
batch_size = 32
learning_rate = 0.001
num_epochs = 10
image_size = 224
csv_file = 'animal_images.csv'  # Path to your CSV file

# Data transformations
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
])

# Read CSV and prepare dataset
df = pd.read_csv('animal_images.csv')

# Convert labels to numerical values and ensure they are integers
label_mapping = {label: idx for idx, label in enumerate(df['label'].unique())}
df['label'] = df['label'].map(label_mapping)
# Split the dataset
train_df, val_df = train_test_split(df, test_size=0.2)

# Create dataset objects
train_dataset = AnimalDataset(dataframe=train_df, transform=transform)
val_dataset = AnimalDataset(dataframe=val_df, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


In [24]:
# Load pre-trained ResNet model and modify it
model = models.resnet50(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

C:\Users\user1\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\user1\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [27]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Training loop
for epoch in range(num_epochs):
    model.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)  # Move data to GPU
        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    total, correct = 0, 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)  # Move data to GPU
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Accuracy: {100 * correct / total}%')

# Save the model
torch.save(model.state_dict(), 'animal_classifier_model.pth')

Epoch [1/10], Accuracy: 45.648464163822524%


KeyboardInterrupt: 